In [2]:
import os
import shutil

creating dumb files 

In [4]:
dir_temp = 'temp'
temp_path = f"{os.getcwd()}/{dir_temp}"

dir_names = ['train', 'valid', 'test']
temp_names = ['ta', 'v', 'ts']

if not os.path.isdir(dir_temp):
    os.mkdir(temp_path)
    for n in dir_names:
        os.mkdir(f"{dir_temp}/{n}")

for index, dir in enumerate(dir_names):
    images_path = f"{dir_temp}/{dir}/images"
    labels_path = f"{dir_temp}/{dir}/labels"
    if not os.path.isdir(images_path):
        os.mkdir(images_path)
        os.mkdir(labels_path)

    for i in range(10):
        image_name = f"{temp_names[index]}_image_{i:02}.txt"
        label_name = f"{temp_names[index]}_label_{i:02}.txt"
        image_path = os.path.join(images_path, image_name)
        label_path = os.path.join(labels_path, label_name)
        open(image_path, mode='x').close()
        open(label_path, mode='x').close()

removing directory contents

In [ ]:
testing_dirs = ['temp', 'dataset']
files = os.listdir(f'{testing_dirs[0]}')

for f in files:
    file_path = f'{testing_dirs[0]}/{f}'
    os.remove(file_path)

remove a directory

In [29]:
dir = ['temp', 'dataset']
shutil.rmtree(dir[0])

copying directory contents to another directory

In [48]:
files = os.listdir('temp')
dst_dir = 'dataset'
for f in files:
    file_to_copy = f"temp/{f}"
    shutil.copy2(file_to_copy, dst_dir)

getting the correct name for directory

In [19]:
def get_dir_name(dir_name):
    if not os.path.isdir(dir_name):
        return dir_name

    path_contents = os.listdir(f"{os.getcwd()}")
    target_dirs = [_ for _ in path_contents if dir_name in _]
    if len(target_dirs) == 1:
        return f'{dir_name}_01'

    last_two_elements = target_dirs[-1][-2:]
    dir_num = int(last_two_elements) + 1

    return f'{dir_name}_{dir_num:02}'

check if dataset could be split

In [20]:
def is_dataset_valid(images_path, labels_path):
    images_numbers, labels_numbers = len(os.listdir(images_path)), len(os.listdir(labels_path))
    if images_numbers == labels_numbers:
        return True
    return False

create the collection of target directory

In [21]:
def create_dir_collection(src_path):
    os.mkdir(src_path)
    images_path, labels_path = f"{src_path}/images", f"{src_path}/labels"
    os.mkdir(images_path)
    os.mkdir(labels_path)

move specific files to a directory

In [8]:
dir_name = 'testing'
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)

junk_files = ['train', 'valid', 'test']
files = [f for f in os.listdir('temp') if not f in junk_files]

for f in files:
    shutil.move(f'temp/{f}', dir_name)

In [10]:
shutil.move(f'temp/train/images/', 'testing')

'testing\\images'

check if directory exist

In [22]:
def is_dir_exist(path):
    return os.path.isdir(path)

decompose directory

In [26]:
def decompose(path):
    if not os.path.exists(path):
        raise Exception("the given path is incorrect!")

    target_dir = ['train', 'valid', 'test']
    
    # check if desire directory exist
    pd_dir = f"{os.getcwd()}/prepared_dataset"
    dst_images_dir = f"{pd_dir}/images"
    dst_labels_dir = f"{pd_dir}/labels"
    if is_dir_exist(pd_dir):
        raise Exception("you must delete 'prepared_dataset folder'!")

    os.mkdir(pd_dir)
    os.mkdir(dst_images_dir)
    os.mkdir(dst_labels_dir)

    for dir in target_dir:
        src_dir = f"{path}/{dir}"
        images_src = f"{src_dir}/images"
        labels_src = f"{src_dir}/labels"
        images_list = os.listdir(images_src)
        labels_list = os.listdir(labels_src)
        for img in images_list:
            shutil.copy2(f"{images_src}/{img}", dst_images_dir)

        for label in labels_list:
            shutil.copy2(f"{labels_src}/{label}", dst_labels_dir)

split function 

In [24]:
def cv_split(raw_dataset_path: str, train_s: int, valid_s: int, test_s: int):
    check_size = train_s + valid_s + test_s
    if check_size != 100:
        raise Exception('make sure that your split data size is incorrect!')

    images_path = os.path.join(raw_dataset_path, 'images')
    labels_path = os.path.join(raw_dataset_path, 'labels')

    if not is_dataset_valid(images_path, labels_path):
        raise Exception('make sure that number of images equal to labels!')

    files = (os.listdir(images_path), os.listdir(labels_path))
    dataset_dir = get_dir_name('dataset')
    os.mkdir(dataset_dir)

    #number of elements
    num_elements = get_elements(files[0], train_s, valid_s, test_s)

    #creating train dir
    train_path = f'{dataset_dir}/train'
    create_dir_collection(train_path)
    setup_train(files, raw_dataset_path, train_path, num_elements['train'])

    #creating valid dir
    valid_path = f'{dataset_dir}/valid'
    create_dir_collection(valid_path)
    valid_index = num_elements['train']
    setup_test_or_valid(files, raw_dataset_path, valid_path, num_elements['valid'], valid_index)

    #creating temp dir
    test_path = f'{dataset_dir}/test'
    create_dir_collection(test_path)
    test_index = num_elements['train'] + num_elements['valid']
    setup_test_or_valid(files, raw_dataset_path, test_path, num_elements['temp'], test_index)


def get_elements(f_list, train_s: int, valid_s: int, test_s: int):
    train_e = len(f_list) * train_s / 100
    if train_e % 1 != 0:
        train_e = int(train_e) + 1
        valid_e = len(f_list) * valid_s // 100
        test_e = len(f_list) * test_s // 100
    else:
        valid_e = len(f_list) * valid_s / 100
        if valid_e % 1 != 0:
            valid_e = int(valid_e) + 1
            test_e = len(f_list) * test_s // 100
        else:
            test_e = len(f_list) * test_s / 100
            if test_e % 1 != 0:
                test_e = int(test_e) + 1

    elements = {
        'train': int(train_e),
        'valid': int(valid_e),
        'temp': int(test_e)
    }
    return elements


def setup_train(files, src_path, dst_path, num_elements):
    src_img_path, dst_img_path = f"{src_path}/images", f"{dst_path}/images"
    src_label_path, dst_label_path = f"{src_path}/labels", f"{dst_path}/labels"
    for image_name, label_name in zip(files[0][:num_elements], files[1][:num_elements]):
        image_to_copy = os.path.join(src_img_path, image_name)
        shutil.copy2(image_to_copy, dst_img_path)  #copy the image 
        label_to_copy = os.path.join(src_label_path, label_name)
        shutil.copy2(label_to_copy, dst_label_path)  #copy the label


def setup_test_or_valid(files, src_path, dst_path, num_elements, start_index):
    src_img_path, dst_img_path = f"{src_path}/images", f"{dst_path}/images"
    src_label_path, dst_label_path = f"{src_path}/labels", f"{dst_path}/labels"

    for image_name, label_name in zip(files[0][start_index: start_index + num_elements],
                                      files[1][start_index: start_index + num_elements]):
        image_to_copy = os.path.join(src_img_path, image_name)
        shutil.copy2(image_to_copy, dst_img_path)  #copy the image 
        label_to_copy = os.path.join(src_label_path, label_name)
        shutil.copy2(label_to_copy, dst_label_path)  #copy the label


In [15]:
decompose('temp')

In [25]:
cv_split('prepared_dataset', 50, 20, 30)

In [ ]:
# move 3 files data.yaml - 2 txt files
# decompose the remaining files - train, test, and valid.